In [3]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase

In [2]:
df=pd.read_csv('gs://calitp-analytics-data/data-analyses/5310/5310_org_info_manualfill.csv')

In [4]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)
bc = bc>>filter(_.funding_program.str.contains('5310'))
bc["organization_name"] = (
    bc["organization_name"]
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace("/", " ")
    #.str.replace("\\", "")
)

/tmp/ipykernel_1043/4035653396.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [5]:
bc.head(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
125,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Amador Transit,BCG0001035,Operating Assistance,300901,64AO18-00677,30144.0,30144.0,30144.0,0.0,0,Open,NaN,NaN,NaN
126,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Angel View Inc,BCG0000324,Operating Assistance,300901,64AO18-00770,125000.0,125000.0,125000.0,0.0,0,Open,NaN,NaN,NaN
127,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0000988,Operating Assistance,300901,64AO18-00630,193211.0,193211.0,193211.0,0.0,0,Open,NaN,NaN,NaN
128,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0002757,Operating Assistance,300901,64AO18-00630,40000.0,40000.0,40000.0,0.0,0,Open,NaN,NaN,NaN
129,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Area 1 Agency on Aging,BCG0000392,Operating Assistance,300901,64AO18-00631,146534.0,146534.0,146534.0,0.0,0,Open,NaN,NaN,NaN


In [7]:
p = pd.merge(bc, df, how='left', on='organization_name')

In [9]:
p['description'] = p['description'].str.strip()

In [11]:
(
    p
    >> group_by(_.organization_name, _.funding_program)
    >> summarize(n=_.description.nunique())
    >> arrange(-_.n)
    >> filter(_.n > 1)
)

,organization_name,funding_program,n
39,Community Bridges Liftline,5310 Trad,6
181,United Cerebral Palsy Association of Greater S...,5310 Trad,6
139,Porterville Sheltered Workshop,5310 Trad,5
11,Asian Community Center of Sacramento Valley In...,5310 Trad,4
21,Catholic Charities of the Diocese of Stockton,5310 Trad,4
...,...,...,...
173,The Center for Independent Living,5310 Trad,2
185,Valley Resource Center,5310 Trad,2
187,Victor Valley Community Services Council,5310 Trad,2
188,Victor Valley Transit Authority,5310 Trad,2


In [12]:
p>>count(_.grant_fiscal_year)

,grant_fiscal_year,n
0,2017,513
1,2019,602


In [13]:
p>>count(_.project_year)

,project_year,n
0,2017,507
1,2018,4
2,2019,604


In [14]:
p>>filter(_.project_year==2018)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,...,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
72,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,County of Sonoma Human Services Department Adu...,BCG0000526,Operating Assistance,300901,64AO18-00794,266975.0,...,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match
73,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,Desert Access and Mobility Inc,BCG0000752,Operating Assistance,300901,64AO16-00777,125000.0,...,NaN,Desert Access and Mobility Inc,NaN,NaN,Non-Profit Organization,NaN,Palm Springs,Riverside,NaN,Name Match; Airtable
510,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000525,Mobility Management,117L00,64AM18-00764,157532.0,...,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match
511,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000598,Purchase Computer Software,114208,64AM18-00764,10000.0,...,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match


In [15]:
p>>count(_.grant_number)


,grant_number,n
0,CA-2017-169 | 0017000186-E,75
1,CA-2017-169 | 0017000186-T,438
2,CA-2020-244 | 0020000273-E,76
3,CA-2020-244 | 0020000273-T,526


In [16]:
(p>>group_by(_.organization_name, _.grant_fiscal_year)>>count(_.description))

,organization_name,grant_fiscal_year,description,n
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,2017,Purchase Replacement Van,4
2,ARC Imperial Valley,2017,Operating Assistance,2
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,4
4,ARC Imperial Valley,2019,Operating Assistance,2
...,...,...,...,...
353,Wilshire Community Services,2019,Operating Assistance,1
354,Work Training Center,2017,Purchase Radios,1
355,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,6
356,Yolo County Transportation District,2017,Mobility Management,2


In [17]:
p>>group_by(_.organization_name)>>count(_.description)

,organization_name,description,n
0,ARC Bakersfield,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,Purchase Replacement Van,4
2,ARC Imperial Valley,Buy Van For Svc Expansion,4
3,ARC Imperial Valley,Operating Assistance,4
4,Able Industries,Buy <30-Ft Bus For Expansion,6
...,...,...,...
280,Wilshire Community Services,Operating Assistance,2
281,Work Training Center,Purchase Radios,1
282,Work Training Center,Purchase Replacement < 30 Ft Bus,6
283,Yolo County Transportation District,Mobility Management,2


In [20]:
p.description.value_counts()

Purchase Replacement < 30 Ft Bus                                                                464
Buy <30-Ft Bus For Expansion                                                                    138
Purchase Replacement Van                                                                        122
Operating Assistance                                                                            113
Buy Van For Svc Expansion                                                                        93
Mobility Management                                                                              72
Purchase Computer Hardware                                                                       16
Purchase Radios                                                                                  16
Surveillance/Security                                                                            10
Support Equip/Facilities-Equipment                                                                8


In [21]:
(
    p
    >> group_by(_.organization_name, _.organization_type)
    >> summarize(nunique_desc=_.description.nunique())
    >> arrange(-_.nunique_desc)
)

,organization_name,organization_type,nunique_desc
28,Community Bridges Liftline,Non-Profit Organization,7
35,County of Sonoma Human Services Department Adu...,Independent Agency,7
136,United Cerebral Palsy Association of Greater S...,Non-Profit Organization,7
85,Mobility Matters,Non-Profit Organization,6
7,Asian Community Center of Sacramento Valley In...,Non-Profit Organization,5
...,...,...,...
135,United Cerebral Palsy Association Inc of Stani...,Non-Profit Organization,1
137,United Cerebral Palsy of San Luis Obispo County,Non-Profit Organization,1
138,Valley Achievement Center,Non-Profit Organization,1
143,West Valley Community Services,Non-Profit Organization,1


In [25]:
p_agg = (p.groupby(['organization_name'])
      .agg({'description': lambda x: x.tolist()})
      .rename({'description' : 'descriptions'},axis=1)
      .reset_index())

In [26]:
p_agg

,organization_name,descriptions
0,ARC Bakersfield,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."
1,ARC Imperial Valley,"[Operating Assistance, Operating Assistance, O..."
2,Able Industries,"[Buy <30-Ft Bus For Expansion, Buy <30-Ft Bus ..."
3,Alegria Community Living,"[Purchase Replacement Van, Purchase Replacemen..."
4,Amador Transit,"[Operating Assistance, Mobility Management, Mo..."
...,...,...
142,Vivalon Inc,"[Operating Assistance, Purchase Replacement < ..."
143,West Valley Community Services,"[Operating Assistance, Operating Assistance, O..."
144,Wilshire Community Services,"[Operating Assistance, Operating Assistance]"
145,Work Training Center,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."


## Specific Project Types
finding project types for 
* wheelchair accessible vehicles
* on-demand scheduling
* dispatching software platforms

In [76]:
p.description.value_counts()

Purchase Replacement < 30 Ft Bus                                                                464
Buy <30-Ft Bus For Expansion                                                                    138
Purchase Replacement Van                                                                        122
Operating Assistance                                                                            113
Buy Van For Svc Expansion                                                                        93
Mobility Management                                                                              72
Purchase Computer Hardware                                                                       16
Purchase Radios                                                                                  16
Surveillance/Security                                                                            10
Support Equip/Facilities-Equipment                                                                8


In [81]:
p>>filter(_.description.str.contains("Buy"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
34,Friends of Children with Special Needs,2017,Buy Van For Svc Expansion
35,Friends of Children with Special Needs,2017,Buy Van For Svc Expansion
75,Able Industries,2017,Buy <30-Ft Bus For Expansion
76,Able Industries,2017,Buy <30-Ft Bus For Expansion
77,Able Industries,2017,Buy <30-Ft Bus For Expansion
...,...,...,...
1080,Valley Achievement Center,2019,Buy <30-Ft Bus For Expansion
1081,Valley Achievement Center,2019,Buy <30-Ft Bus For Expansion
1082,Valley Achievement Center,2019,Buy <30-Ft Bus For Expansion
1083,Valley Achievement Center,2019,Buy <30-Ft Bus For Expansion


In [82]:
p>>filter(_.description.str.contains("Purchase"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
85,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus
86,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus
87,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus
88,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus
89,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus
...,...,...,...
1110,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1111,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1112,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1113,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus


In [95]:
p>>filter(_.description.str.contains("On-demand"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description


In [97]:
p>>filter(_.description.str.contains("Operating Assistance"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
0,Amador Transit,2017,Operating Assistance
1,Angel View Inc,2017,Operating Assistance
2,ARC Imperial Valley,2017,Operating Assistance
3,ARC Imperial Valley,2017,Operating Assistance
4,Area 1 Agency on Aging,2017,Operating Assistance
...,...,...,...
584,United Cerebral Palsy Association of Greater S...,2019,Operating Assistance
585,Valley Resource Center,2019,Operating Assistance
586,West Valley Community Services,2019,Operating Assistance
587,West Valley Community Services,2019,Operating Assistance


In [98]:
p>>filter(_.description.str.contains("Software"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
117,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Software
118,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Software
508,Yolo County Transportation District,2017,Purchase Computer Software
509,Yolo County Transportation District,2017,Purchase Computer Software
511,County of Sonoma Human Services Department Adu...,2018,Purchase Computer Software
631,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Software
632,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Software


In [99]:
p>>filter(_.description.str.contains("Hardware"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
111,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Hardware
166,Community Bridges Liftline,2017,Purchase Computer Hardware
231,Foothill AIDS Project,2017,Purchase Computer Hardware
421,San Joaquin Regional Transit District,2017,Purchase Computer Hardware
477,United Cerebral Palsy Association of Greater S...,2017,Purchase Computer Hardware
478,United Cerebral Palsy Association of Greater S...,2017,Purchase Computer Hardware
629,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Hardware
630,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Hardware
653,Community Bridges Liftline,2019,Purchase Computer Hardware
654,Community Bridges Liftline,2019,Purchase Computer Hardware


In [33]:
p_agg2 = (p.groupby(['organization_name', 'project_year'])
      .agg({'description': lambda x: x.tolist()})
      .rename({'description' : 'descriptions'},axis=1)
      .reset_index())

In [53]:
p_agg2

,organization_name,project_year,descriptions
0,ARC Bakersfield,2017,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."
1,ARC Imperial Valley,2017,"[Operating Assistance, Operating Assistance]"
2,ARC Imperial Valley,2019,"[Operating Assistance, Operating Assistance, B..."
3,Able Industries,2017,"[Buy <30-Ft Bus For Expansion, Buy <30-Ft Bus ..."
4,Alegria Community Living,2019,"[Purchase Replacement Van, Purchase Replacemen..."
...,...,...,...
221,West Valley Community Services,2019,"[Operating Assistance, Operating Assistance, O..."
222,Wilshire Community Services,2017,[Operating Assistance]
223,Wilshire Community Services,2019,[Operating Assistance]
224,Work Training Center,2017,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."


In [57]:
 year = p.groupby(['organization_name','project_year']).agg({'description': lambda x: x.tolist()}).unstack('project_year', fill_value=0)

In [58]:
year

description  \
project_year                                                                      2017   
organization_name                                                                        
ARC Bakersfield                      [Purchase Replacement < 30 Ft Bus, Purchase Re...   
ARC Imperial Valley                       [Operating Assistance, Operating Assistance]   
Able Industries                      [Buy <30-Ft Bus For Expansion, Buy <30-Ft Bus ...   
Alegria Community Living                                                             0   
Amador Transit                             [Operating Assistance, Mobility Management]   
...                                                                                ...   
Vivalon Inc                          [Operating Assistance, Purchase Replacement < ...   
West Valley Community Services                                                       0   
Wilshire Community Services                                     [Operating Assistance]   
Work Training Center                 [Purchase Replacement < 30 Ft Bus, Purchase Re...   
Yolo County Transportation District  [Mobility Management, Mobility Management, Pur...   

                                          \
project_year                        2018   
organization_name                          
ARC Bakersfield                        0   
ARC Imperial Valley                    0   
Able Industries                        0   
Alegria Community Living               0   
Amador Transit                         0   
...                                  ...   
Vivalon Inc                            0   
West Valley Community Services         0   
Wilshire Community Services            0   
Work Training Center                   0   
Yolo County Transportation District    0   

                                                                                        
project_year                                                                      2019  
organization_name                                                                       
ARC Bakersfield                                                                      0  
ARC Imperial Valley                  [Operating Assistance, Operating Assistance, B...  
Able Industries                                                                      0  
Alegria Community Living             [Purchase Replacement Van, Purchase Replacemen...  
Amador Transit                              [Mobility Management, Mobility Management]  
...                                                                                ...  
Vivalon Inc                          [Purchase Replacement < 30 Ft Bus, Purchase Re...  
West Valley Community Services       [Operating Assistance, Operating Assistance, O...  
Wilshire Community Services                                     [Operating Assistance]  
Work Training Center                                                                 0  
Yolo County Transportation District                                                  0  

[147 rows x 3 columns]

In [62]:
# testing for single organization: Amador Transit
(p>>filter(_.organization_name =='Amador Transit'))>>select(_.project_year, _.description)

,project_year,description
0,2017,Operating Assistance
81,2017,Mobility Management
599,2019,Mobility Management
600,2019,Mobility Management


In [65]:
# great, exporting
#year.to_csv('by_years.csv')